In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv(r'D:\customer_churn_project\data\telco_Customer_Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
df.shape

(7043, 21)

In [4]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')#change the object to numeric_value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [6]:
df.dropna(inplace=True)
df.shape

(7032, 21)

In [7]:
df['Churn'].value_counts(normalize =True)*100 ## Calculate the percentage of customers who churned vs. stayed

Churn
No     73.421502
Yes    26.578498
Name: proportion, dtype: float64

In [8]:
 df['tenure_buckets']=pd.cut(df['tenure'],
                     bins=[0,6,12,24,100],
                     labels=['0-6 months','6-12 months','1-2 years','2+years']
                     )
df[['tenure','tenure_buckets']].head(10)

,tenure,tenure_buckets
0,1,0-6 months
1,34,2+years
2,2,0-6 months
3,45,2+years
4,2,0-6 months
5,8,6-12 months
6,22,1-2 years
7,10,6-12 months
8,28,2+years
9,62,2+years


In [9]:
pd.crosstab(df['tenure_buckets'],df['Churn'],normalize='index')*100 # Group customers into tenure buckets (0-6m, 6-12m, 1-2y, 2+y) for analysis

Churn,No,Yes
tenure_buckets,,
0-6 months,46.666667,53.333333
6-12 months,64.113475,35.886525
1-2 years,71.289062,28.710938
2+years,85.963997,14.036003


In [10]:
df['revenue_at_risk'] = np.where(
    df['Churn'] == 'Yes',
    df['MonthlyCharges'],
    0
)
def churn_risk_score(row):
    score=0
    if row['Contract']=='Month-to-month':
        score+=40
    if row['tenure']<6:
        score+=30
    if row['MonthlyCharges']>70:
        score+=20
    if row['PaymentMethod']=='Electronic check':
        score+=10
        
    return score
    
    

In [11]:
df['churn_risk_score'] = df.apply(churn_risk_score, axis=1)

In [12]:
df[['Contract','tenure','MonthlyCharges','PaymentMethod','churn_risk_score']].head()


,Contract,tenure,MonthlyCharges,PaymentMethod,churn_risk_score
0,Month-to-month,1,29.85,Electronic check,80
1,One year,34,56.95,Mailed check,0
2,Month-to-month,2,53.85,Mailed check,70
3,One year,45,42.30,Bank transfer (automatic),0
4,Month-to-month,2,70.70,Electronic check,100


In [13]:
df['risk_category']=pd.cut(
    df['churn_risk_score'],
    bins=[0,30,60,100],
    labels=['low_risk','medium_risk','high_risk']
)
final_df = df[
    ['customerID', 'tenure', 'Contract', 'PaymentMethod',
     'MonthlyCharges', 'Churn', 'revenue_at_risk',
     'churn_risk_score', 'risk_category']
]
final_df

,customerID,tenure,Contract,PaymentMethod,MonthlyCharges,Churn,revenue_at_risk,churn_risk_score,risk_category
0,7590-VHVEG,1,Month-to-month,Electronic check,29.85,No,0.00,80,high_risk
1,5575-GNVDE,34,One year,Mailed check,56.95,No,0.00,0,NaN
2,3668-QPYBK,2,Month-to-month,Mailed check,53.85,Yes,53.85,70,high_risk
3,7795-CFOCW,45,One year,Bank transfer (automatic),42.30,No,0.00,0,NaN
4,9237-HQITU,2,Month-to-month,Electronic check,70.70,Yes,70.70,100,high_risk
...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,24,One year,Mailed check,84.80,No,0.00,20,low_risk
7039,2234-XADUH,72,One year,Credit card (automatic),103.20,No,0.00,20,low_risk
7040,4801-JZAZL,11,Month-to-month,Electronic check,29.60,No,0.00,50,medium_risk
7041,8361-LTMKD,4,Month-to-month,Mailed check,74.40,Yes,74.40,90,high_risk


In [14]:
final_df.to_csv("../data/churn_final.csv", index=False)
